In [ ]:
#@title Stable Diffusion web-ui colab от [Meldoner](https://github.com/Meldoner)
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
!git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
#@markdown Выбор готовой модели
model = "sd2.1" #@param ["openjourneyv2","sd2.1","sd1.5","abyss-orange-mix2","anythingv3","anythingv4.5","deliberate","dreamlike","dreamshaper","kenshi","protogenx5.8","grapefruit","realistic-vision","corneos-7th-heaven-mix","pastel-mix"]

#@markdown Нажми, если ты хочешь загрузить свою модель
load_own_model = False #@param{type:"boolean"}
model_url = "Вставь ссылку на модель (ckpt/safetensors)" #@param {type:"string"}

#@markdown Нажми, если ты хочешь использовать свой VAE
load_own_vae = False #@param{type:"boolean"}
vae_url = "Вставь ссылку на VAE" #@param {type:"string"}

#@markdown Нажми, если ты хочешь использовать свои embeddings
load_own_embedding = False #@param{type:"boolean"}
embedding_url = "Вставь ссылку на embedding" #@param {type:"string"}

#@markdown Нажми, если ты хочешь использовать свою LORA
load_own_lora = False #@param{type:"boolean"}
lora_url = "Вставь ссылку на LORA" #@param {type:"string"}

#@markdown Вы можете загрузить несколько моделей(и остальное), просто перезапусти скрипт

#@markdown Если ты хочешь, чтобы я добавил больше моделей, создай запрос [здесь](https://github.com/Meldoner/stable-diffusion-webui-colab/pulls)

if (model=='openjourneyv2') and (load_own_model == False):
  !wget -nc "https://huggingface.co/prompthero/openjourney-v2/resolve/main/openjourney-v2.ckpt" -P /content/stable-diffusion-webui/models/Stable-diffusion

if (model=='abyss-orange-mix2') and (load_own_model == False):
  !wget -nc "https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_nsfw.safetensors" -P /content/stable-diffusion-webui/models/Stable-diffusion
  !wget -nc "https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt" -P /content/stable-diffusion-webui/models/VAE

if (model=='anythingv3') and (load_own_model == False):
  !wget -nc "https://huggingface.co/admruul/anything-v3.0/resolve/main/Anything-V3.0-pruned.safetensors" -P /content/stable-diffusion-webui/models/Stable-diffusion
  !wget -nc "https://huggingface.co/admruul/anything-v3.0/resolve/main/Anything-V3.0.vae.pt" -P /content/stable-diffusion-webui/models/VAE

if (model=='anythingv4.5') and (load_own_model == False):
  !wget -nc "https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned.safetensors" -P /content/stable-diffusion-webui/models/Stable-diffusion
  !wget -nc "https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0.vae.pt" -P /content/stable-diffusion-webui/models/VAE

if (model=='deliberate') and (load_own_model == False):
  !wget -nc "https://huggingface.co/XpucT/Deliberate/resolve/main/Deliberate.safetensors" -P /content/stable-diffusion-webui/models/Stable-diffusion

if (model=='dreamlike') and (load_own_model == False):
  !wget -nc "https://huggingface.co/dreamlike-art/dreamlike-diffusion-1.0/resolve/main/dreamlike-diffusion-1.0.safetensors" -P /content/stable-diffusion-webui/models/Stable-diffusion

if (model=='dreamshaper') and (load_own_model == False):
  !wget -nc "https://huggingface.co/Lykon/DreamShaper/resolve/main/Dreamshaper_3.32_baked_vae_clip_fix.safetensors" -P /content/stable-diffusion-webui/models/Stable-diffusion

if (model=='kenshi') and (load_own_model == False):
  !wget -nc "https://huggingface.co/SweetLuna/Kenshi/resolve/main/KENSHI%2001/KENSHI01_Pruned.safetensors" -P /content/stable-diffusion-webui/models/Stable-diffusion
  !wget -nc "https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt" -P /content/stable-diffusion-webui/models/VAE
  
if (model=='protogenx5.8') and (load_own_model == False):
  !wget -nc "https://huggingface.co/darkstorm2150/Protogen_x5.8_Official_Release/resolve/main/ProtoGen_X5.8.safetensors" -P /content/stable-diffusion-webui/models/Stable-diffusion
  
if (model=='sd2.1') and (load_own_model == False):
  !wget -nc "https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors" -P /content/stable-diffusion-webui/models/Stable-diffusion
  
if (model=='sd1.5') and (load_own_model == False):
  !wget -nc "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors" -P /content/stable-diffusion-webui/models/Stable-diffusion
  
if (model=='grapefruit') and (load_own_model == False):
  !wget -nc "https://huggingface.co/iZELX1/Grapefruit/resolve/main/grapefruitv4.safetensors" -P /content/stable-diffusion-webui/models/Stable-diffusion
  !wget -nc "https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0.vae.pt" -P /content/stable-diffusion-webui/models/VAE
    
if (model=='realistic-vision') and (load_own_model == False):
  !wget -nc "https://huggingface.co/SG161222/Realistic_Vision_V1.3/resolve/main/Realistic_Vision_V1.3.safetensors" -P /content/stable-diffusion-webui/models/Stable-diffusion
    
if (model=='corneos-7th-heaven-mix') and (load_own_model == False):
  !wget -nc "https://huggingface.co/dwarfbum/corneos7thHeavenMix_v2/resolve/main/corneos7thHeavenMix_v2.safetensors" -P /content/stable-diffusion-webui/models/Stable-diffusion
  !wget -nc "https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0.vae.pt" -P /content/stable-diffusion-webui/models/VAE

if (model=='pastel-mix') and (load_own_model == False):
  !wget -nc "https://huggingface.co/andite/pastel-mix/resolve/main/pastelmix.safetensors" -P /content/stable-diffusion-webui/models/Stable-diffusion
  !wget -nc "https://huggingface.co/andite/pastel-mix/resolve/main/pastel-waifu-diffusion.vae.pt" -P /content/stable-diffusion-webui/models/VAE

if  load_own_model == True:
  !wget -nc "$model_url" -P /content/stable-diffusion-webui/models/Stable-diffusion

if load_own_vae == True:
  !wget -nc "$vae_url" -P /content/stable-diffusion-webui/models/VAE

if load_own_embedding == True:
  !wget -nc "$embedding_url" -P /content/stable-diffusion-webui/embeddings

if load_own_lora == True:
  !wget -nc "$lora_url" -P /content/stable-diffusion-webui/models/Lora

!pip install --upgrade fastapi==0.90.1
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117
%cd /content/stable-diffusion-webui
!COMMANDLINE_ARGS="--share --disable-safe-unpickle --no-half-vae --xformers --reinstall-xformers --enable-insecure-extension-access" REQS_FILE="requirements.txt" python launch.py